In [10]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
import numpy as np
import pandas as pd

In [13]:
data = pd.read_csv("../processed/processed-shutoffs.csv")
data.head()

,circuit_name,deenergize_time,restoration_time,time_out_min,key_communities,hftd_tier,total_affected,residential_affected,zip_code,longitude,latitude,substn_present,total_population
0,APPLE HILL-1103,2018-10-14 21:00:00,2018-10-15 16:29:00.000000000,1169.0,"Camino, Placerville",3.0,1809.0,1488.0,95709,-120.677165,38.739683,True,5634.0
1,APPLE HILL-1104,2018-10-14 21:17:00,2018-10-15 18:37:00.000000000,1280.0,"Camino, Placerville, Pollock Pines,",3.0,2261.0,2043.0,95709,-120.677165,38.739683,True,5634.0
2,APPLE HILL-2102,2018-10-14 21:05:00,2018-10-16 18:04:00.000000000,2699.0,"Camino, Grizzly Flats, Mount Aukum, Placervill...",3.0,4489.0,4013.0,95709,-120.677165,38.739683,True,5634.0
3,BONNIE NOOK-1102,2018-10-14 21:54:00,2018-10-15 18:04:00.000000000,1210.0,Alta,3.0,533.0,454.0,95701,-120.809292,39.214708,True,1064.0
4,CALISTOGA-1101,2018-10-14 20:34:00,2018-10-15 21:27:00.000000000,1493.0,Calistoga,3.0,1596.0,1216.0,94515,-122.578610,38.592088,True,7525.0


In [14]:
data["denergize_time"] = pd.to_datetime(data["deenergize_time"])
data["restoration_time"] = pd.to_datetime(data["restoration_time"])
data["deenergize_date"] = data["denergize_time"].dt.date
data["restoration_date"] = data["restoration_time"].dt.date


In [ ]:
missing = []
hourly = {}
daily = {}
for i in range(data.shape[0]):
    point_1 = Point(data.loc[i, "latitude"], data.loc[i, "longitude"])
    start = data.loc[i, "denergize_time"] - timedelta(days=2)
    end = data.loc[i, "restoration_time"]
    weather = Hourly(point_1, start, end)
    weather = weather.fetch()
    if weather.shape[0] == 0:
        weather = Daily(point_1, start, end)
        weather = weather.fetch()
        if weather.shape[0] == 0:
            missing.append(i)
        else:
            daily[i] = weather
    else:
        hourly[i] = weather

In [ ]:
# missing = pd.DataFrame(missing)
# missing = missing.rename(columns={0: "index"})
# missing["Circuit Name"] = data.iloc[missing["index"], :].reset_index()[
#     "circuit_name"]
# missing.to_csv("missing_weather.csv")


In [50]:
# for i in list(daily.keys()):
#     shutoff = data.iloc[i, :]
#     filename = f"raw/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}_daily.csv"
#     weather = daily[i]
#     weather.to_csv(filename)
    

In [51]:
# for i in list(hourly.keys()):
#     shutoff = data.iloc[i, :]
#     filename = f"raw/weather_{shutoff['circuit_name']}_{shutoff['deenergize_date']}_hourly.csv"
#     weather = hourly[i]
#     weather.to_csv(filename)
